In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns

from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
dataset =pd.read_csv('/kaggle/input/insurance/insurance.csv')
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.isnull().any()

**Data Analysis**

In [ ]:
categorical_features=[feature for feature in dataset.columns if dataset[feature].dtypes=='O']
for feature in categorical_features:
    sns.countplot(dataset[feature])
    plt.title(feature)
    plt.show()

In [ ]:
corr=dataset.corr()
cont_features=corr.index
sns.heatmap(dataset[cont_features].corr(), annot=True, cmap='RdYlGn')
plt.title('Corellation')

In [ ]:
sns.pairplot(dataset)

**Feature Engineering**

In [ ]:
Y=dataset['charges']
X=dataset.copy()
X=X.drop(['charges'], axis=1)
X.head()

In [ ]:
Y.head()

In [ ]:
for column in categorical_features:
        dummy=pd.get_dummies(X[column], drop_first=True);
        X=X.drop(column, axis=1)
        X=pd.concat([X, dummy], axis=1)
X.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler().fit(X)
X=sc.transform(X)

**Model Building and Execution**

In [ ]:
def build_model(hp):
    model=keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
            model.add(layers.Dense(units=hp.Int('units_'+str(i), min_value=32, max_value=512, step=32), activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer= keras.optimizers.Adam(
    hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
            loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model;




In [ ]:
tuner=RandomSearch(build_model,
                  objective='val_mean_absolute_error',
                  max_trials=5,
                  executions_per_trial=3)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(X, Y, test_size=0.3)

In [ ]:
tuner.search(x_train, y_train, validation_split=0.1, epochs=5)

In [ ]:
best_model=tuner.get_best_models(num_models=1)[0]
y_predict=best_model.predict(x_test)


from sklearn.metrics import r2_score
print(r2_score(y_test, y_predict))